In [1]:
import os
import gc
import random

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torch import optim

import numpy as np
import pandas as pd

from glob import glob
from tqdm import tqdm#_notebook as tqdm

from experiment import experiment, load_experiment
from models.BaseModel import BaseModel
from butterflyDataset import butterflyDataset
from experimentUtils import get_model_called, get_loss_function_called, get_optimizer_called, get_transform_called

from utils import plot, check_memory_usage, start_timer, tick, count_parameters

import matplotlib
import matplotlib.pyplot as plt

%matplotlib nbagg
%matplotlib inline

cuda = torch.cuda.is_available()

In [10]:
ex.avg_epoch_duration

Timedelta('0 days 00:00:08.129399')

In [2]:
config = dict(modelName='EvenSimplerAutoEncoder',
              modelParams=dict(imgSize=(160,240),hidden_features=500,latent_features=100, in_channels=3),
                 max_num_epochs = 2,
                 suffix='_delete',
                 opt='adam',
                 optParams = {'lr': 0.0001},
                 loss='mse',
             batchSize=35,
             earlyStopping=15,
             sides='D', 
             transforms=[('resize', {'size': (160, 240)}), 
                         'hflip', 
                         ('colorjitter',{'contrast':0.1,'saturation':0.1}), 
                         ('affine',dict(degrees=2, scale=(0.8,1.2), fillcolor=(255,255,255))),
                         'totensor'])

ex = experiment(**config)
#ex.prep_experiment()
ex.test_experiment()
ex.run_experiment()

Prepping Experiment..
Testing data throughput...
<class 'PIL.Image.Image'>
<class 'PIL.Image.Image'>
<class 'PIL.Image.Image'>
<PIL.Image.Image image mode=RGB size=960x640 at 0x7F3736F02DD8>
<PIL.Image.Image image mode=RGB size=960x640 at 0x7F3736F02DD8>
<PIL.Image.Image image mode=RGB size=960x640 at 0x7F3736F02DD8>
<class 'PIL.Image.Image'>
<class 'PIL.Image.Image'>
<PIL.Image.Image image mode=RGB size=960x640 at 0x7F3736F02DD8>
<PIL.Image.Image image mode=RGB size=960x640 at 0x7F373052F358>
<PIL.Image.Image image mode=RGB size=960x640 at 0x7F3730466438>
<class 'PIL.Image.Image'>
<class 'PIL.Image.Image'>
<PIL.Image.Image image mode=RGB size=960x640 at 0x7F3736F02DD8>
<class 'PIL.Image.Image'>
<PIL.Image.Image image mode=RGB size=960x640 at 0x7F3730133FD0>
<class 'PIL.Image.Image'>
<PIL.Image.Image image mode=RGB size=960x640 at 0x7F373030B518>


TypeError: Traceback (most recent call last):
  File "/home/rob/anaconda3/envs/thesis/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 138, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/home/rob/anaconda3/envs/thesis/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 138, in <listcomp>
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/home/rob/Dropbox/thesis/2. code/src/butterflyDataset.py", line 56, in __getitem__
    sample = self.transform(sample)
  File "/home/rob/anaconda3/envs/thesis/lib/python3.7/site-packages/torchvision/transforms/transforms.py", line 49, in __call__
    img = t(img)
  File "/home/rob/Dropbox/thesis/2. code/src/myTransforms.py", line 19, in __call__
    return cv2.resize(img, self.size)
TypeError: Expected cv::UMat for argument 'src'


In [6]:
ex.classifier_t_accuracy

0.8148148148148148

In [4]:
config = dict(modelName='BasicAutoEncoderNoMaxUnpool',
              modelParams=dict(imgSize=(160,240),hidden_features=500,latent_features=100, in_channels=3),
                 max_num_epochs = 300,
                 suffix='_100lf_V_aug',
                 opt='adam',
                 optParams = {'lr': 0.0005},
                 loss='mse',
             batchSize=30,
             earlyStopping=15,
             sides='V', 
             transforms=[('resize', {'size': (160, 240)}), 
                         'hflip', 
                         ('colorjitter',{'contrast':0.1,'saturation':0.1}), 
                         ('affine',dict(degrees=2, scale=(0.8,1.2), fillcolor=(255,255,255))),
                         'totensor'])

ex = experiment(**config)
#ex.prep_experiment()
ex.test_experiment()
ex.run_experiment()

No classifier improvement, stopping after 577 epochs
tensor(0.8372, grad_fn=<NllLossBackward>)


In [14]:
(np.array(ex.y_train) == 'UNCLEAR').mean()

0.025893958076448828

In [6]:
config = dict(modelName='VariationalAutoEncoderNoUnpool',
              modelParams=dict(imgSize=(160,240),hidden_features=500,latent_features=100,num_samples=15, cuda=True, in_channels=3),
                 max_num_epochs = 300,
                 suffix='_100lf_V_aug',
                 opt='adam',
                 optParams = {'lr': 0.0001},
                 loss='elbo',
             batchSize=7,
             earlyStopping=15,
             sides='V', 
             transforms=[('resize', {'size': (160, 240)}), 
                         'hflip', 
                         ('colorjitter',{'contrast':0.1,'saturation':0.1}), 
                         ('affine',dict(degrees=2, scale=(0.8,1.2), fillcolor=(255,255,255))),
                         'totensor'])

ex = experiment(**config)
#ex.prep_experiment()
ex.test_experiment()
ex.run_experiment()

No classifier improvement, stopping after 39 epochs
tensor(0.9430, grad_fn=<NllLossBackward>)
